In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, plot_roc_curve, make_scorer
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
import time
pd.set_option("display.max_columns", 20)
pd.set_option("display.max_rows", 250)

df_train = pd.read_csv("../input/santander-customer-transaction-prediction/train.csv")
df_test = pd.read_csv("../input/santander-customer-transaction-prediction/test.csv")

In [ ]:
features = ['var_' + str(i) for i in range(0,199)]
X = df_train[features]
y = df_train['target']

In [ ]:
start_time = time.time()

cv = StratifiedKFold(n_splits=5, shuffle=False)

#XG boost with gridsearch


param_grid_xgb ={'n_estimators':[20000], 'learning_rate':[0.01],'gamma':[0], 'scale_pos_weight':[1], 'reg_alpha':[0], 'min_child_weight':[1], 'subsample':[0.8], 'colsample_bytree': [0.8], 'max_depth':[3], 'tree_method':['gpu_hist'], 'gpu_id':[0]}
grid_search_xgb = GridSearchCV(XGBClassifier(objective='binary:logistic'), param_grid=param_grid_xgb, cv=cv, scoring='roc_auc', n_jobs=-1)
grid_result = grid_search_xgb.fit(X,y)
print(grid_search_xgb.best_params_)
print(grid_search_xgb.best_score_)


print('Process finished in ', time.time() - start_time, ' s')

In [ ]:
#Catboost with gridsearch
from catboost import CatBoostClassifier

start_time = time.time()


param_grid_cat ={'learning_rate':[0.3], 'iterations':[500], 'loss_function':['Logloss'], 'depth':[3], 'verbose':[True], 'task_type':['GPU'],'devices':['0']}
grid_search_cat = GridSearchCV(CatBoostClassifier(), param_grid=param_grid_cat, cv=cv, scoring='roc_auc', n_jobs=-1)
grid_search_cat.fit(X,y)
print(grid_search_cat.best_params_)
print(grid_search_cat.best_score_)

print('Process finished in ', time.time() - start_time, ' s')

In [ ]:
#voting classifier

from sklearn.ensemble import VotingClassifier
start_time = time.time()

models = {'CatBoost': grid_search_cat, 'XGBoost': grid_search_xgb}
majority_vote_clf = VotingClassifier([VotingClassifier([(name, model.best_estimator_) for name, model in models.items()], voting='soft')
print('Majority Vote: ', cross_val_score(majority_vote_clf, X, y, cv=cv, n_jobs=-1).mean())
                                      
print('Process finished in ', time.time() - start_time, ' s')

In [ ]:
#stacking 
start_time = time.time()


from sklearn.ensemble import StackingClassifier
models = {'CatBoost': grid_search_cat, 'XGBoost': grid_search_xgb}
stacking_clf = StackingClassifier([(name, model.best_estimator_) for name, model in models.items()])
print('Stacking Classifier: ', cross_val_score(stacking_clf, X, y, cv=cv, n_jobs=-1).mean())

print('Process finished in ', time.time() - start_time, ' s')

In [ ]:
#submission file
stacking_clf.fit(X, y)

X_test = df_test[features]
y_pred = stacking_clf.predict_proba(X_test)

df_submission = pd.DataFrame()
df_submission['ID_code'] = df_test['ID_code']
df_submission['target'] = y_pred[:,1]

df_submission.to_csv('submission - stacking.csv', index=False)
